# 第5章 深度学习计算

## 5.1 层和块

### 练习 5.1.1 

如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？ 

**解答：** 

如果将MySequential中存储块的方式更改为Python列表，则会失去原有的顺序保持和快速查找的优势。同时，需要手动处理块的顺序和输入数据的传递，增加代码的复杂度和容易出错。因此，建议仍然使用OrderedDict来存储块。

### 练习 5.1.2

实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。

**解答：**

在本书7.4节中GoogleLet模型中的Inception块使用了平行块技术.
下面代码实现了一个并行网络，由两个子网络组成。输入数据先分别经过两个子网络的计算，分别得到两个部分的输出结果，然后在通道维度上合并结果得到最终输出。其中，net1和net2分别表示两个子网络，Sequential表示将多个层组成一个序列的容器，Linear表示一个线性层，ReLU表示一个激活函数，cat表示在指定维度上拼接张量。最后，输出结果的大小是(2, 36)，表示有2个样本，每个样本的特征维度是36。

In [4]:
import torch.nn as nn
import torch


class Parallel(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1=net1 # 第一个子网络
        self.net2=net2 # 第二个子网络
        
    def forward(self,X):
        x1= self.net1(X) # 第一个子网络的输出
        x2= self.net2(X) # 第二个子网络的输出
        return torch.cat((x1,x2),dim=1) # 在通道维度上合并输出结果
      
X = torch.rand(2,10) # 输入数据
net = Parallel(nn.Sequential(nn.Linear(10,12),nn.ReLU()), nn.Sequential(nn.Linear(10,24),nn.ReLU())) # 实例化并行网络
output = net(X) # 前向传播
print(output.size()) # 输出结果的大小

torch.Size([2, 36])


### 练习 5.1.3

假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

**解答：** 

In [5]:
def create_network(num_instances, input_size, hidden_size, output_size):
    # 创建一个线性层
    linear_layer = nn.Sequential(
        nn.Linear(input_size, hidden_size), nn.ReLU(),
        nn.Linear(hidden_size, input_size)
    )
    
    # 创建多个实例并连接
    instances = [linear_layer for _ in range(num_instances)]
    network = nn.Sequential(*instances)
    
    # 添加输出层
    output_layer = nn.Linear(input_size, output_size)
    network.add_module("output", output_layer)
    
    return network
# 示例用法
net = create_network(num_instances=3, input_size=10, hidden_size=5, output_size=2)
net

Sequential(
  (0): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (output): Linear(in_features=10, out_features=2, bias=True)
)

## 5.2 参数管理 

### 练习 5.2.1

使用`FancyMLP`模型，访问各个层的参数。

 **解答：**

引用上5.1节中的FancyMLP模型,并用state_dict()方法访问模型的全部参数。 

In [7]:
from torch.nn import functional as F
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

X = torch.rand(2, 20)
net = FixedHiddenMLP()
net(X)
net.state_dict()

### 练习 5.2.2

查看初始化模块文档以了解不同的初始化方法。

**解答：**

- `torch.nn.init.uniform_(tensor, a=0.0, b=1.0)`：使输入的张量服从（a,b）的均匀分布并返回。

- `torch.nn.init.normal_(tensor, mean=0.0, std=1.0)`：从给定的均值和标准差的正态分布N(mean,std2)中生成值，初始化张量。

- `torch.nn.init.constant_(tensor, val)`：以一确定数值初始化张量。

- `torch.nn.init.xavier_uniform_(tensor, gain=1.0)`：从均匀分布U(−a, a)中采样，初始化输入张量，其中a的值由如下公式确定

  $a= gain * \sqrt{\frac{6}{fan_{in}+fan_{out}}}$

  其中的`gain`为可缩放因子，可以用`torch.nn.init.calculate_gain(nonlinearity, param=None)`方法得到，此方法其实就是一查表，背后对应的表格如下

  ![初始化方法-gain因子.png](../.././images/初始化方法-gain因子.png)

- `torch.nn.init.xavier_normal_(tensor, gain=1.0)`:从正态分布N(0,std2)中采样，初始化输入张量，其中std值由下式确定：

  $a= gain * \sqrt{\frac{2}{fan_{in}+fan_{out}}}$

- `torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从均匀分布U(−bound, bound)，其中bound值由下式确定

  $bound= gain * \sqrt{\frac{3}{fan_{mode}}}$

- `torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从从正态分布N(0,std2)中采样，其中std值由下式确定

  $std= \frac{gain}{\sqrt{fan_{mode}}}$



### 练习 5.2.3

构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

**解答：** 

在训练过程中，我们每个epoch都打印了每层的参数和梯度。可以看到shared_fc层的参数和梯度都是相同的，因为它们共享同一个参数。

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# 模型参数
input_size = 2
hidden_size = 4
output_size = 2
lr = 0.01
epochs = 2

# 构建带有共享参数层的多层感知机
shared_fc = nn.Linear(hidden_size, hidden_size)
MLP = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(),
                    shared_fc, nn.ReLU(),
                    shared_fc, nn.ReLU(),
                    nn.Linear(hidden_size, output_size)
)

# 训练数据
X = torch.randn(1, input_size)
Y = torch.randn(1, output_size)
# 创建模型和优化器
# MLP(input_size, hidden_size, output_size)
optimizer = optim.SGD(MLP.parameters(), lr=lr)
# 训练模型
for epoch in range(epochs):
    # 前向传播和计算损失
    Y_pred = MLP(X)
    loss = nn.functional.mse_loss(Y_pred, Y)
    # 反向传播和更新梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 打印每层的参数和梯度
    for name, param in MLP.named_parameters():
        print(name, param.data, param.grad)
    print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))

0.weight tensor([[-0.1180,  0.1194],
        [ 0.5747, -0.4074],
        [ 0.2131,  0.1560],
        [ 0.5971, -0.5614]]) tensor([[-0.0000, -0.0000],
        [-0.1484, -0.1379],
        [-0.0000, -0.0000],
        [-0.0861, -0.0800]])
0.bias tensor([-0.2144,  0.2941,  0.0637,  0.3801]) tensor([0.0000, 0.2614, 0.0000, 0.1518])
2.weight tensor([[ 0.3504,  0.1069,  0.0545,  0.0266],
        [ 0.0044,  0.4416,  0.4497,  0.4842],
        [-0.3265,  0.2675, -0.2384, -0.3932],
        [-0.2787,  0.2753,  0.1946,  0.2054]]) tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.1166,  0.0831,  0.3548],
        [ 0.0000,  0.0602,  0.0529,  0.2004],
        [ 0.0000,  0.0106, -0.0276, -0.0283]])
2.bias tensor([-0.4919, -0.2057,  0.1845,  0.1653]) tensor([ 0.0000,  1.1674,  0.6683, -0.1243])
6.weight tensor([[-0.2431,  0.4902,  0.3570, -0.3329],
        [-0.4766, -0.4757, -0.2619,  0.0098]]) tensor([[ 0.0000,  0.0135,  0.0514,  0.1942],
        [-0.0000, -0.0151, -0.0573, -0.2167]])
6

### 练习 5.2.4

为什么共享参数是个好主意？

**解答：** 

1. 节约内存：共享参数可以减少模型中需要存储的参数数量，从而减少内存占用。

2. 加速收敛：共享参数可以让模型更加稳定，加速收敛。

3. 提高泛化能力：共享参数可以帮助模型更好地捕捉数据中的共性，提高模型的泛化能力。

4. 加强模型的可解释性：共享参数可以让模型更加简洁明了，加强模型的可解释性。 

## 5.3 延后初始化 

### 练习 5.3.1 

如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？

**解答：** 

可以正常运行。第一层会立即初始化,但其他层是直到数据第一次通过模型传递才会初始化。

In [15]:
import torch
from torch import nn

"""延后初始化"""
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
# print(net[0].weight)  # 尚未初始化
print(net)

X = torch.rand(2, 20)
net(X)
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)
Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


D:\anaconda\anaconda_install\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


### 练习 5.3.2

如果指定了不匹配的维度会发生什么？

**解答：** 

会由于矩阵乘法的维度不匹配而报错。

### 练习 5.3.3 

如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。

**解答：**

如果输入维度比指定维度小，可以考虑使用padding填充；如果输入维度比指定维度大，可以考虑用pca等降维方法，将维度降至指定维度。再或者我们可以改变模型的指定维度。

## 5.4 自定义层 

### 练习 5.4.1 

设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。

**解答：** 

In [17]:
import torch.nn as nn

class TensorLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(TensorLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        y = torch.einsum('bij,bik->bjk', [x.unsqueeze(1), x.unsqueeze(2)])
        y = torch.einsum('bij,jik->bjk', [y.view(batch_size, -1), self.weight])
        return y

### 练习 5.4.2 

设计一个返回输入数据的傅立叶系数前半部分的层。

**解答：** 

In [18]:
import torch.nn as nn
import torch.fft as fft

class FourierLayer(nn.Module):
    def __init__(self):
        super(FourierLayer, self).__init__()

    def forward(self, x):
        x = fft.fftn(x)
        x = x[:, :, :x.shape[2] // 2]
        return x

## 5.5 读写文件 

###  练习 5.5.1

即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

**解答：** 

1. 加速/避免重复训练：可用于在训练模型过程中生成checkout文件（存储模型参数），必要时可以go back，方便后续梯度下降时找到局部最优点。

2. 便于共享和复制：可以将参数保存到文件中，并将文件发送给其他人，而无需共享整个模型。

### 练习 5.5.2

假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

**解答：** 

使用保存模型某层参数的办法，保存网络的前两层，然后再加载到新的网络中使用。

```python
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')
clone = MLP()
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))
print(clone.hidden.weight == net.hidden.weight)
```

### 练习 5.5.3

如何同时保存网络架构和参数？需要对架构加上什么限制？

**解答：** 

pytorch本身不提供同时保存网络架构和参数的方法，这点和TensorFlow不同。需要确保在保存模型之前，已经定义了网络对象，因为模型参数只能加载到与其形状相同的网络中。

In [20]:
import torch.nn as nn
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# 存储模型
torch.save(net.state_dict(), 'model.pt')

# 导入模型
model = Net()
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

## 5.6 GPU 

### 练习 5.6.1 

尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？

**解答：** 

计算量很大的任务：使用GPU速度明显更快

计算量很小的任务：CPU速度可能更快，因为数据传输到GPU需要时间

### 练习 5.6.2

我们应该如何在GPU上读写模型参数？

**解答：** 

```python
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.load_state_dict(torch.load(PATH))
````

### 练习 5.6.3 

测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。

**解答:** 

In [23]:
import torch
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 生成随机矩阵
matrices = [torch.randn(100, 100).to(device) for i in range(1000)]

# 计算时间
start_time = time.time()
for i in range(1000):
    result = torch.mm(matrices[i], matrices[i].t())
    frobenius_norm = torch.norm(result)
#     print(frobenius_norm)
end_time = time.time()

# print("Time taken:", end_time - start_time)

### 练习 5.6.4

测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

**解答：** 

In [ ]:
import torch
import time

# 创建两个随机矩阵
a = torch.randn(10000, 10000).cuda()
b = torch.randn(10000, 10000).cuda()

# 顺序计算
start_time = time.time()
c1 = torch.matmul(a, b)
c2 = torch.matmul(a, b)
end_time = time.time()
sequential_time = end_time - start_time

print(f"Sequential time: {sequential_time:.4f} seconds")

# 检查GPU数量
print(f"Number of GPUs: {torch.cuda.device_count()}")

# 分配GPU
device1 = torch.device("cuda:0")
device2 = torch.device("cuda:1")

# 存到不同GPU
a = torch.randn(10000, 10000).to(device1)
b = torch.randn(10000, 10000).to(device2)

# 并行计算
start_time = time.time()
a1, a2 = torch.chunk(a, 2, dim=1)
b1, b2 = torch.chunk(b, 2, dim=0)
c1 = torch.matmul(a1, b1) + torch.matmul(a2, b2)
c2 = torch.matmul(a1, b1) + torch.matmul(a2, b2)
end_time = time.time()
parallel_time = end_time - start_time

print(f"Parallel time: {parallel_time:.4f} seconds")